In [1]:
import pandas as pd
import csv

In [2]:
glencore = pd.read_csv('glencorePR.csv', delimiter = ',')

In [3]:
glencore = glencore.drop(['_source.title', '_score', '_source.doctype', '_type', '_index', '_source.url', '_source.feedurl'], axis = 1)
glencore = glencore.rename(index=str, columns={"_source.title_rss": "title", "_source.teaser_rss": "content"})

In [4]:
contents = []
contents = glencore.content.astype(str)
#to extract content of PR release, and tokenize

In [5]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [9]:
texts_lower = [[word for word in document.lower().split()] for document in contents]

In [12]:
from nltk.tokenize import word_tokenize
texts_tokenized = [[word.lower() for word in word_tokenize(document)] for document in contents]

In [13]:
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

In [18]:
texts_nstopwords = [[word for word in document if not word in english_stopwords] for document in texts_tokenized]
print(texts_nstopwords[0])

['<', '!', '[', 'cdata', '[', '<', 'p', '>', 'glencore', 'plc', 'today', 'announced', 'board', 'recommending', 'aggregate', 'distribution', 'us', '$', '0.20', 'per', 'share', 'respect', '2017', 'financial', 'year', 'ended', '31', 'december.', '<', '/p', '>', '<', 'p', '>', 'distributions', 'declared', 'paid', 'u.s.', 'dollars', ',', 'although', 'shareholders', 'jersey', 'register', 'able', 'elect', 'receive', 'distribution', 'payments', 'pounds', 'sterling', ',', 'euros', 'swiss', 'francs', '.', 'shareholders', 'hold', 'shares', 'jersey', 'register', ',', 'computershare', 'hk', 'nominee', ',', 'receive', 'distribution', 'payment', 'hong', 'kong', 'dollars', ',', 'converted', 'jersey', 'applicable', 'exchange', 'rate', 'reference', 'date', '.', 'shareholders', 'johannesburg', 'register', 'receive', 'distributions', 'south', 'african', 'rand.', '<', '/p', '>', '<', 'p', '>', 'subject', 'shareholder', 'approval', ',', 'distribution', 'made', 'capital', 'contribution', 'reserves', 'company

In [29]:
english_punctuations = [',', '.', ':', ';', '?', '(', ')', '&', '!', '*', '@', '#', '$', '%', '>', '<', '[' , ']']
texts_filtered = [[word for word in document if not word in english_punctuations] for document in texts_nstopwords]

In [31]:
from nltk.stem.lancaster import LancasterStemmer

In [32]:
st = LancasterStemmer()

In [33]:
st.stem('stemmed')

'stem'

In [34]:
texts_stemmed = [[st.stem(word) for word in docment] for docment in texts_filtered]

In [36]:
all_stems = sum(texts_stemmed, [])

In [37]:
stems_once = set(stem for stem in set(all_stems) if all_stems.count(stem) == 1)

In [38]:
texts = [[stem for stem in text if stem not in stems_once] for text in texts_stemmed]

In [39]:
from gensim import corpora, models, similarities

In [40]:
import logging

In [41]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [42]:
dictionary = corpora.Dictionary(texts)

2018-08-21 14:13:12,825 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-21 14:13:12,895 : INFO : built Dictionary(2213 unique tokens: ["''", '+41', '/', '//www.glencore.com', '/a']...) from 136 documents (total 50635 corpus positions)


In [43]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [44]:
 tfidf = models.TfidfModel(corpus)

2018-08-21 14:13:51,790 : INFO : collecting document frequencies
2018-08-21 14:13:51,792 : INFO : PROGRESS: processing document #0
2018-08-21 14:13:51,805 : INFO : calculating IDF weights for 136 documents and 2212 features (21358 matrix non-zeros)


In [45]:
corpus_tfidf = tfidf[corpus]

In [46]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)

2018-08-21 14:15:01,617 : INFO : using serial LSI version on this node
2018-08-21 14:15:01,619 : INFO : updating model with new documents
2018-08-21 14:15:01,884 : INFO : preparing a new chunk of documents
2018-08-21 14:15:01,908 : INFO : using 100 extra samples and 2 power iterations
2018-08-21 14:15:01,909 : INFO : 1st phase: constructing (2213, 110) action matrix
2018-08-21 14:15:01,925 : INFO : orthonormalizing (2213, 110) action matrix
2018-08-21 14:15:02,087 : INFO : 2nd phase: running dense svd on (110, 136) matrix
2018-08-21 14:15:02,120 : INFO : computing the final decomposition
2018-08-21 14:15:02,123 : INFO : keeping 10 factors (discarding 43.074% of energy spectrum)
2018-08-21 14:15:02,136 : INFO : processed documents up to #136
2018-08-21 14:15:02,146 : INFO : topic #0(5.477): 1.000*"nan" + 0.000*"+41" + -0.000*"/" + 0.000*"''" + 0.000*"td" + 0.000*"valign=" + 0.000*"/index/who-we-are/our-leadership" + 0.000*"width=" + 0.000*"top" + 0.000*"/tr"
2018-08-21 14:15:02,148 : IN

In [49]:
index = similarities.MatrixSimilarity(lsi[corpus])

2018-08-21 14:21:04,685 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-08-21 14:21:04,718 : INFO : creating matrix with 136 documents and 10 features


In [52]:
example = texts[0]
exbow = dictionary.doc2bow(example)
ex_lsi = lsi[exbow]
print(ex_lsi)

[(1, 94.36106668820437), (2, 34.075254712069956), (3, 121.68830756940838), (4, 62.01745165033903), (5, 37.76573343242043), (6, -2.020686107228405), (7, 18.742595001332326), (8, -4.37501005460515), (9, 3.2656228890751366)]


In [53]:
sims = index[ex_lsi]
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [55]:
print(sort_sims[0:10])

[(0, 1.0), (25, 1.0), (40, 0.99014145), (16, 0.97376674), (7, 0.9725734), (12, 0.97251433), (8, 0.8888424), (26, 0.6752852), (96, 0.6752852), (49, 0.6571536)]
